# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [13]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-0bb0244e63-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Horror Franchises explorative search") 

Consider the following exploratory scenario:


> we are interested in horror movies, in particular we care about the slasher subgenre, and we want to compare the Halloween and the Friday the 13th francise. 

Background knowledge useful for better conducting the search: Halloween is a movie from 1978 with many sequels and the set of Halloween + all the sequels is called the "Halloween franchise". The same considerations hold for Friday the 13th.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q221103`     | Halloween       | node |
| `wd:Q1243029`| Friday the 13th       | node |





Also consider

```
wd:Q1243029 ?p ?obj .
```

is the BGP to retrieve all **properties of Friday the 13th**

The workload should


1. Investigate the workers of the two original movies and check any commonality 

2. Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)

3. Check which actor/actress did more movies after the release of these two films (consider the films separately)

4. Get all the movies of the two franchises (return year and director)

5. Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)? 

6. Investigate which franchise is highly regarded by the critics overall

7. Investigate which franchise costed more 

In [3]:
# start your workflow here

In [4]:
queryString = """
SELECT *
WHERE { 

wd:Q1243029 ?p ?obj .
} 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.gov/authorities/names/no98033420')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3212'), ('obj', 'urn:isan:0000-0000-31EA-0000-G-0000-0000-Q')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P4276'), ('obj', 'https://data.cinematheque.qc.ca/data/Work1467')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/016mng')]


5

### Task 1. Investigate the workers of the two original movies and check any commonality 


I search for common properties of the two films that links the workers with the movies

In [5]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname WHERE {
     wd:Q221103 ?p ?o .
    
    # this returns the labels
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor'), ('o', 'http://www.wikidata.org/entity/Q574983'), ('oname', 'Tommy Lee Wallace')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('pname', 'movement'), ('o', 'http://www.wikidata.org/entity/Q377616'), ('oname', 'New Hollywood')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q130232'), ('oname', 'drama')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q200092'), ('oname', 'horror film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q1146335'), ('oname', 'teen film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q853630'), ('oname', 'slasher film')]
[('p', 'http://www.wikidata.org/prop/direct/P1434'), ('pname', 'takes place in fictional universe'

60

In [16]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname WHERE {
     wd:Q1243029 ?p ?o .
    
    # this returns the labels
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q1067324'), ('oname', 'exploitation film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q1200678'), ('oname', 'mystery film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q853630'), ('oname', 'slasher film')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q909586'), ('oname', 'splatter film')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q1780634'), ('oname', 'Golden Raspberry Award for Worst Picture')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q977915'), ('oname', 'Golden Raspberry Award for Worst Supporting Actress')]
[('p', 'ht

43

I need to check the property of human taking one random character from above

In [6]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname WHERE {
     wd:Q273574 ?p ?o .
    
    # this returns the labels
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10800557'), ('oname', 'film actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q3282637'), ('oname', 'film producer')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q10798782'), ('oname', 'television actor')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation'), ('o', 'http://www.wikidata.org/entity/Q33999'), ('oname', 'actor')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth'), ('o', 'http://www.wikidata.org/entity/Q47164'), ('oname', 'Santa Monica')]
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender'), ('o', 'http://www.wikidata.org/entity/Q6581072'), ('oname', 'female')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pname', 'father'), ('o

10

I see that Q5 is the code for humans

In [7]:
queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname WHERE {
     wd:Q221103 ?p ?o .
     ?o wdt:P31 wd:Q5 .
    
    # this returns the labels
    ?o <http://schema.org/name> ?oname .
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q2865227'), ('oname', 'Arthur Malet')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q3066766'), ('oname', 'Tony Moran')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q106997'), ('oname', 'Jamie Lee Curtis')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q201570'), ('oname', 'Nancy Kyes')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor'), ('o', 'http://www.wikidata.org/entity/Q574983'), ('oname', 'Tommy Lee Wallace')]
[('p', 'http://www.wikidata.org/prop/direct/P2554'), ('pname', 'production designer'), ('o', 'http://www.wikidata.org/entity/Q574983'), ('oname', 'Tommy Lee Wallace')]
[('p', 'http://www.wikidata.org/prop/direct/P1

25

Properties I am interested in, relatively to Halloween are: film editor (P1040), P161 (cast member), producer (P162), performer(P175), director of photography (P344), screenwriter(P58), composer(P86), production designer (P2554), executive producer(P1431)

let's check for Friday the 13th if there is something in common

In [15]:
queryString = """

    SELECT DISTINCT ?p ?pname ?o ?oname WHERE {
        wd:Q221103 ?p ?o .
        wd:Q1243029 ?p ?o .
        ?o wdt:P31 wd:Q5 .

        # this returns the label                 
        ?p <http://schema.org/name> ?pname .
        ?o <http://schema.org/name> ?oname .  
}
"""

print("Results")
run_query(queryString)

Results
Empty


0

No commonality

### Task 2. Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)

Check properties of an actor, for example let's choose Jamie Lee Curtis, that acted in Halloween

In [8]:
queryString = """        #halloween 
SELECT DISTINCT ?p ?pName ?o ?oName WHERE { 
        wd:Q106997 ?p ?o .
                     
        ?p <http://schema.org/name> ?pName .
        ?o <http://schema.org/name> ?oName .  
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P734'), ('pName', 'family name'), ('o', 'http://www.wikidata.org/entity/Q12787162'), ('oName', 'Curtis')]
[('p', 'http://www.wikidata.org/prop/direct/P22'), ('pName', 'father'), ('o', 'http://www.wikidata.org/entity/Q162389'), ('oName', 'Tony Curtis')]
[('p', 'http://www.wikidata.org/prop/direct/P25'), ('pName', 'mother'), ('o', 'http://www.wikidata.org/entity/Q205314'), ('oName', 'Janet Leigh')]
[('p', 'http://www.wikidata.org/prop/direct/P3373'), ('pName', 'sibling'), ('o', 'http://www.wikidata.org/entity/Q273574'), ('oName', 'Kelly Curtis')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q1185037'), ('oName', 'Beverly Hills High School')]
[('p', 'http://www.wikidata.org/prop/direct/P69'), ('pName', 'educated at'), ('o', 'http://www.wikidata.org/entity/Q5103452'), ('oName', 'Choate Rosemary Hall')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pName', 'nomi

47

I decide to make the most famous actor or actress based on the number of award nominations.

In [9]:
queryString = """        
SELECT ?actorName (COUNT (?o) as ?numNomination) WHERE {
{
     wd:Q221103 wdt:P161 ?actor .
         ?actor wdt:P1411 ?o .
        
        # this returns the label                 
        ?actor <http://schema.org/name> ?actorName .  
}
UNION
{ 
         wd:Q1243029 wdt:P161 ?actor .
         ?actor wdt:P1411 ?o .
        
        # this returns the label                 
        ?actor <http://schema.org/name> ?actorName .  
}
}
ORDER BY desc (?numNomination)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('actorName', 'Jamie Lee Curtis'), ('numNomination', '9')]


1

### Task 3. Check which actor/actress did more movies after the release of these two films (consider the films separately)


I see that in both of the films there isn't any property related to the year of release of the movie. I had to remove this constraint.

Firstly, Halloween

In [18]:
queryString = """        
SELECT ?actorName (COUNT (?film) as ?filmCount) WHERE {
        wd:Q221103 wdt:P161 ?actor .
        ?film wdt:P161 ?actor .
        
        # this returns the label
        ?actor <http://schema.org/name> ?actorName . 
}
GROUP BY ?actorName
ORDER BY desc(?filmCount)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('actorName', 'Donald Pleasence'), ('filmCount', '128')]


1

Then, Friday the 13th

In [19]:
queryString = """        
SELECT ?actorName (COUNT (?film) as ?filmCount) WHERE {
        wd:Q1243029 wdt:P161 ?actor .
        ?film wdt:P161 ?actor .
        
        # this returns the label
        ?actor <http://schema.org/name> ?actorName . 
}
GROUP BY ?actorName
ORDER BY desc(?filmCount)
LIMIT 1
"""

print("Results")
run_query(queryString)

Results
[('actorName', 'Kevin Bacon'), ('filmCount', '69')]


1

### Task 4. Get all the movies of the two franchises (return year and director)

Firstly, Halloween

In [77]:
queryString = """        
SELECT ?s ?film ?director WHERE 
{
        ?s wdt:P179 wd:Q1364022 ;
            wdt:P57 ?d .
       
        # this returns the label
        ?s <http://schema.org/name> ?film .
        ?d <http://schema.org/name> ?director .
}
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q959853'), ('film', 'Halloween 4: The Return of Michael Myers'), ('director', 'Dwight H. Little')]
[('s', 'http://www.wikidata.org/entity/Q203705'), ('film', 'Halloween III: Season of the Witch'), ('director', 'Tommy Lee Wallace')]
[('s', 'http://www.wikidata.org/entity/Q850312'), ('film', 'Halloween II'), ('director', 'John Carpenter')]
[('s', 'http://www.wikidata.org/entity/Q221103'), ('film', 'Halloween'), ('director', 'John Carpenter')]
[('s', 'http://www.wikidata.org/entity/Q908690'), ('film', 'Halloween II'), ('director', 'Rob Zombie')]
[('s', 'http://www.wikidata.org/entity/Q909063'), ('film', 'Halloween'), ('director', 'Rob Zombie')]
[('s', 'http://www.wikidata.org/entity/Q1050815'), ('film', 'Halloween H20: 20 Years Later'), ('director', 'Steve Miner')]
[('s', 'http://www.wikidata.org/entity/Q976728'), ('film', 'Halloween 5: The Revenge of Michael Myers'), ('director', 'Dominique Othenin-Girard')]
[('s', 'http://www.wikidata.org/e

11

In [ ]:
Then, Friday the 13th

In [79]:
queryString = """        
SELECT ?s ?film ?director WHERE 
{
        ?s wdt:P179 wd:Q1138878 ;
            wdt:P57 ?d .
       
        # this returns the label
        ?s <http://schema.org/name> ?film .
        ?d <http://schema.org/name> ?director .
}
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q1454799'), ('film', 'Friday the 13th Part VI: Jason Lives'), ('director', 'Tom McLoughlin')]
[('s', 'http://www.wikidata.org/entity/Q1427521'), ('film', 'Jason X'), ('director', 'James Isaac')]
[('s', 'http://www.wikidata.org/entity/Q1243009'), ('film', 'Friday the 13th Part 2'), ('director', 'Sean S. Cunningham')]
[('s', 'http://www.wikidata.org/entity/Q1243029'), ('film', 'Friday the 13th'), ('director', 'Sean S. Cunningham')]
[('s', 'http://www.wikidata.org/entity/Q1243009'), ('film', 'Friday the 13th Part 2'), ('director', 'Steve Miner')]
[('s', 'http://www.wikidata.org/entity/Q1243013'), ('film', 'Friday the 13th Part III'), ('director', 'Steve Miner')]
[('s', 'http://www.wikidata.org/entity/Q1454798'), ('film', 'Friday the 13th Part VII: The New Blood'), ('director', 'John Carl Buechler')]
[('s', 'http://www.wikidata.org/entity/Q965859'), ('film', 'Freddy vs. Jason'), ('director', 'Ronny Yu')]
[('s', 'http://www.wikidata.org/entity/

13

I check again if I can find the year of publication of the movies

In [15]:
queryString = """
SELECT DISTINCT ?pname ?o WHERE {
     wd:Q221103 ?p ?o .
    
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('pname', 'film editor'), ('o', 'http://www.wikidata.org/entity/Q574983')]
[('pname', 'Box Office Mojo film ID (former scheme)'), ('o', 'halloween')]
[('pname', 'Rotten Tomatoes ID'), ('o', 'm/1009113-halloween')]
[('pname', 'AlloCiné film ID'), ('o', '9434')]
[('pname', 'movement'), ('o', 'http://www.wikidata.org/entity/Q377616')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q130232')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q200092')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q1146335')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q853630')]
[('pname', 'Encyclopædia Britannica Online ID'), ('o', 'topic/Halloween-1978-film')]
[('pname', 'takes place in fictional universe'), ('o', 'http://www.wikidata.org/entity/Q101480110')]
[('pname', 'title'), ('o', 'Halloween')]
[('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q850312')]
[('pname', 'AllMovie title ID'), ('o', 'v21317')]
[('pname', 'cast member

128

In [16]:
queryString = """
SELECT DISTINCT ?pname ?o WHERE {
     wd:Q1243029 ?p ?o .
    
    # this returns the labels
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('pname', 'Box Office Mojo film ID (former scheme)'), ('o', 'friday13th')]
[('pname', 'Rotten Tomatoes ID'), ('o', 'm/friday_the_13th_part_1')]
[('pname', 'AlloCiné film ID'), ('o', '505')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q1067324')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q1200678')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q853630')]
[('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q909586')]
[('pname', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q1780634')]
[('pname', 'nominated for'), ('o', 'http://www.wikidata.org/entity/Q977915')]
[('pname', 'title'), ('o', 'Friday the 13th')]
[('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q1243009')]
[('pname', 'AllMovie title ID'), ('o', 'v18658')]
[('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q2440012')]
[('pname', 'cast member'), ('o', 'http://www.wikidata.org/entity/Q263765')]
[('pname', 'cast member'), ('o', 'h

112

I don't find anything related to the year of publication.


### Task 5. Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)?




In [28]:
queryString = """
SELECT (COUNT (?per) as ?numFilms) ?person ?occupation WHERE {
        
    ?per wdt:P31 wd:Q5 .
    ?movie ?p ?per ;
             wdt:P179 ?franchise .
    ?per wdt:P106 ?occ .
    
        
    # this returns the labels
    ?per <http://schema.org/name> ?person .
    ?movie <http://schema.org/name> ?film .
    ?occ <http://schema.org/name> ?occupation .
    
    FILTER ((?franchise = wd:Q1138878) || (?franchise = wd:Q1364022)) .
}
GROUP BY ?person ?occupation
ORDER BY desc(?numFilms)
LIMIT 1

"""

print("Results")
run_query(queryString)

Results
[('numFilms', '15'), ('person', 'Harry Manfredini'), ('occupation', 'film score composer')]


1

### Task 6.Investigate which franchise is highly regarded by the critics overall


I search in various film of the two franchises a property linked to the review scores, taking the codes from the first query of the point 4.

In [6]:
queryString = """
SELECT DISTINCT ?p ?pname ?o WHERE {
     wd:Q959853 ?p ?o .
     
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)'), ('o', 'halloween4')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID'), ('o', 'm/halloween_4_the_return_of_michael_myers')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID'), ('o', '26222')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q200092')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q853630')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title'), ('o', 'Halloween 4: The Return of Michael Myers')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q203705')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by'), ('o', 'http://www.wikidata.org/entity/Q976728')]
[('p', 'ht

86

I can't find anything, also with the film checked below

### Task 7.Investigate which franchise costed more

In [ ]:
queryString = """
SELECT DISTINCT ?p ?pname ?o WHERE {
     wd:Q1454794 ?p ?o .
     
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor'), ('o', 'http://www.wikidata.org/entity/Q17679551')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)'), ('o', 'friday13th8')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID'), ('o', 'm/friday_the_13th_part_8_jason_takes_manhattan')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID'), ('o', '54258')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q200092')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title'), ('o', 'Friday the 13th Part VIII: Jason Takes Manhattan')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pname', 'logo image'), ('o', 'http://commons.wikimedia.org/wiki/Special:FilePath/Vienes%2013%202009%20logo.jpg')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follow

78

In [8]:

queryString = """
SELECT DISTINCT ?p ?pname ?o WHERE {
     wd:Q203705 ?p ?o .
     
    ?p <http://schema.org/name> ?pname .
}
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)'), ('o', 'halloween3')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID'), ('o', 'm/halloween_3_season_of_the_witch')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID'), ('o', '59978')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q200092')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q471839')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre'), ('o', 'http://www.wikidata.org/entity/Q853630')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title'), ('o', 'Halloween III: Season of the Witch')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows'), ('o', 'http://www.wikidata.org/entity/Q850312')]
[('p', 'http://www.wikidata.or

101

I found P2769

Halloween saga

In [10]:
queryString = """
SELECT DISTINCT ?o WHERE {
     ?f wdt:P2769 ?o .
     ?f wdt:P179 wd:Q1364022 .
    
    # this returns the labels
    ?f <http://schema.org/name> ?movie .
}
"""

print("Results")
run_query(queryString)

Results
[('o', '15000000')]
[('o', '2500000')]
[('o', '3000000')]
[('o', '5000000')]


4

Friday the 13th saga

In [11]:
queryString = """
SELECT DISTINCT ?o WHERE {
     ?f wdt:P2769 ?o .
     ?f wdt:P179 wd:Q1138878 .
    
    # this returns the labels
    ?f <http://schema.org/name> ?movie .
}
"""

print("Results")
run_query(queryString)

Results
Empty


0

I can't compare since I have too few data on the budget of the movies.